In [15]:
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    HfArgumentParser,
    TapexTokenizer, BartForConditionalGeneration
)
import os
import sys
import pandas as pd
from io import StringIO

src_path = os.path.abspath(os.path.join(os.path.dirname("__file__"), "../src"))
datasets_path = os.path.abspath(os.path.join(os.path.dirname("__file__"), "../datasets"))

if src_path not in sys.path:
    sys.path.append(src_path)

if datasets_path not in sys.path:
    sys.path.append(datasets_path)

from parsers.argument_classes import DatasetArguments
from utils.datasets_loader import load_datasets

In [16]:
def _convert_csv_string_to_table(csv_string: str) -> pd.DataFrame:
    """
    Convert a csv string to a table, including the header
    """
    df = pd.read_csv(StringIO(csv_string), delimiter=",", on_bad_lines="skip")
    columns = df.columns.astype(str).tolist()
    columns = [col.replace("Unnamed: 0", "") for col in columns]
    rows = df.values.astype(str).tolist()
    return pd.DataFrame(rows, columns = columns)

def main():
    parser = HfArgumentParser(DatasetArguments)
    dataset_args = parser.parse_dict({
    "dataset_root_dir" : "../datasets",
    "train_max_samples_for_each_dataset" : 10,
    "dataset_names": ["wtq"],
    })[0]
    
    # Load datasets
    def filter_function(example):
        if dataset_args.max_table_row_num is not None and example["table_row_num"] > dataset_args.max_table_row_num:
            return False
        if dataset_args.max_table_width is not None and example["table_width"] > dataset_args.max_table_width:
            return False
        return True
    datasets = load_datasets(dataset_args, filter_function=filter_function)
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained("microsoft/tapex-large-finetuned-wtq")
    
    # Model
    model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/tapex-large-finetuned-wtq")
    question = datasets["test"]['question'][0]
    table = datasets["test"]["table"][0]

    df = _convert_csv_string_to_table(table)
    encoding = tokenizer(df, question, padding='max_length', truncation=True, return_tensors="pt")

    outputs = model.generate(**encoding)
    predicted_answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    print("Question:", question)
    print("Answer:", predicted_answer)


In [17]:
main()

/Users/yejuahn/opt/anaconda3/envs/cs230/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Question: did dmitry mikhailovich golitsyn or andrey kirillovich razumovsky serve as ambassador longer?
Answer: [' dmitry mikhailovich golitsyn']
